# Analyze movement based on Accelerometer Data

In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from collections import namedtuple
from itertools import compress, product
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version
from scipy import signal, stats

# import datetime as dt
# #mne
# import mne_bids
# import mne


In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')


In [ ]:
os.chdir(codepath)

# own functions
import lfpecog_features.moveDetection_preprocess as movePrep
import lfpecog_features.moveDetection_run as run_tap_detect
import lfpecog_features.moveDetection_pausedTapFinder as findTap

import lfpecog_analysis.load_SSD_features as load_ssd_fts
import lfpecog_analysis.get_acc_task_derivs as accDerivs

import utils.utils_fileManagement as utilsFiles
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_plotting.plotHelpers as plotHelp

In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', plt_version)
# Python sys 3.9.0 (default, Nov 15 2020, 08:30:55) [MSC v.1916 64 bit (AMD64)]
# pandas 1.4.4
# numpy 1.23.3
# matplotlib 3.5.3

# STATS developed with:
# Python sys 3.10.8 (main, Nov  4 2022, 13:42:51) [MSC v.1916 64 bit (AMD64)]
# pandas 1.5.3
# numpy 1.26.2
# matplotlib 3.8.2

### 0. Define available Subjects


In [ ]:
import lfpecog_analysis.movement_psd_analysis as movePSD
import lfpecog_features.feats_helper_funcs as ftHelpers
import lfpecog_analysis.prep_movement_psd_analysis as prepMovePSD

import lfpecog_features.get_ssd_data as ssd
import lfpecog_analysis.ft_processing_helpers as ftProc


In [ ]:
FT_VERSION='v6'
SETTINGS = utilsFiles.load_ft_ext_cfg(FT_VERSION=FT_VERSION)

main_data_path = os.path.join(get_project_path('data'),
                              'merged_sub_data',
                              SETTINGS["DATA_VERSION"])

SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=SETTINGS["DATA_VERSION"],
                                     FT_VERSION=FT_VERSION)
print(len(SUBS))


## 1. Spectral analysis - split on movement-task-dyskinesia

In [ ]:
sub = '019'

sub_data_path = os.path.join(get_project_path('data'),
                             'merged_sub_data',
                             SETTINGS["DATA_VERSION"],
                             f'sub-{sub}')
fname = (f'{sub}_mergedData_{SETTINGS["DATA_VERSION"]}'
        '_acc_left.P')  # side does not matter for already detected bool labels

# load Acc-detected movement labels
accl = load_class_pickle(os.path.join(sub_data_path, fname))
accl = correct_acc_class(accl)

fname = (f'{sub}_mergedData_{SETTINGS["DATA_VERSION"]}'
        '_acc_right.P')  # side does not matter for already detected bool labels

# load Acc-detected movement labels
accr = load_class_pickle(os.path.join(sub_data_path, fname))
accr = correct_acc_class(accr)



In [ ]:
print(accl.data.shape, accl.fs, accl.colnames)
print(accr.data.shape, accr.fs, accr.colnames)

In [ ]:

# call from feats_extract_multivar.py
ssd_sub = ssd.get_subject_SSDs(
    sub=sub,
    incl_stn=True,
    incl_ecog=False,
    ft_setting_fname='ftExtr_spectral_v6.json',)

In [ ]:
# # get boolean arrays for movement labels corresponding to ephys
# ephys_arr = ssd_sub.lfp_left.lo_beta
# ephys_wintimes = np.array(ssd_sub.lfp_left.times)
# ephys_fs = ssd_sub.lfp_left.fs
# WINLEN_SEC = 10

# # create timestamps for every ephys sample in 2d array (2048 Hz)
# ephys_times2d = np.array([
#     np.arange(t, t + WINLEN_SEC, 1 / ephys_fs)
#     for t in ephys_wintimes
# ])
# nan_arr = np.isnan(ephys_arr)

# # get movement bools based on acc data (512 Hz)
# MOVE_BOOLS = {'no_move': accl.data[:, -1],
#               'any_move': np.sum(accl.data[:, -5:-1], axis=1) > 0,
#               'left_tap': accl.data[:, -5],
#               'left_allmove': (accl.data[:, -5] + accl.data[:, -3]) > 0,
#               'right_tap': accl.data[:, -4],
#               'right_allmove': (accl.data[:, -4] + accl.data[:, -2]) > 0}
# TASK = accl.data[:, 4]

In [ ]:
# CAT = False
# cdrs = ftProc.find_select_nearest_CDRS_for_ephys(
#     sub=sub, side='bilat',  
#     ft_times=ephys_wintimes / 60,
#     INCL_CORE_CDRS=True,
#     cdrs_rater='Patricia',
# )
 
# if CAT:
#     cdrs = ftProc.categorical_CDRS(cdrs,
#                             cutoff_mildModerate=3.5,
#                             cutoff_moderateSevere=7.5,)

In [ ]:
# importlib.reload(prepMovePSD)

# cdrs = ftProc.find_select_nearest_CDRS_for_ephys(
#     sub=sub, side='bilat',  
#     ft_times=ephys_wintimes / 60,
#     INCL_CORE_CDRS=True,
#     cdrs_rater='Patricia',
# )
# # cdrs = ftProc.categorical_CDRS(cdrs,
# #                             cutoff_mildModerate=3.5,
# #                             cutoff_moderateSevere=7.5,)

# mask_times = prepMovePSD.get_mask_timings(
#     orig_labels=cdrs,
#     orig_times=ephys_wintimes,
#     MASK='LID')

# # currently codes: rest=0, tap=1, free=2
# lid_mask = prepMovePSD.create_ephys_mask(
#     ephys_time_arr=ephys_times2d,
#     ephys_win_times=ephys_wintimes,
#     mask_times=mask_times,
#     MASK='LID',
# )

In [ ]:
import lfpecog_analysis.psd_analysis_classes as psdClass
import lfpecog_analysis.specific_ephys_selection as ephySel

import lfpecog_plotting.plot_descriptive_SSD_PSDs as plot_ssd_PSDs
import lfpecog_plotting.plot_move_spec_psd as plot_specPsd
import lfpecog_analysis.prep_stats_movLidspecPsd as prep_specStats
import lfpecog_analysis.psd_lid_stats as psd_Stats

import lfpecog_features.bursts_funcs as bursts_funcs

In [ ]:
import lfpecog_predict.prepare_predict_arrays as predArrays


In [ ]:
importlib.reload(ephySel)
importlib.reload(prepMovePSD)
importlib.reload(psdClass)
importlib.reload(ssd)

# # get single subClass (incl original 3d data, always creates new)
# test = psdClass.PSD_vs_Move_sub(sub='014')

# use existing
sub = '017'
data_path = 'D://Research/CHARITE/projects/dyskinesia_neurophys/data/'
picklepath = os.path.join(
    data_path,
    'windowed_data_classes_10s_0.5overlap',
    'selected_ephys_classes_all'
)
picklename = f'ephys_selections_{sub}.P'

sub_class = load_class_pickle(
    file_to_load=os.path.join(picklepath,
                                picklename),
    convert_float_np64=True
)

Check MASKs with ACC comparison

In [ ]:
importlib.reload(prepMovePSD)

move_masks, task_mask, lid_masks = prepMovePSD.create_ephys_masks(sub='012', verbose=True)

In [ ]:
for MOV_TYPE in move_masks.keys():
    match_sum = np.sum(np.logical_and(
        move_masks[MOV_TYPE] == 1,
        task_mask == 2
    ))
    print(f'{MOV_TYPE} during FREE: {match_sum / 2048} seconds')


In [ ]:
accl = accDerivs.get_raw_acc_traces(sub='012', side='left', data_version='v4.0')
print(accl.colnames)

#### Check and Plot extracted PSD data

Add STATS for PSD plotting

In [ ]:
# CREATE AND/OR PLOT SIGNIFICANCIES PER SELECTION (binary / linear)
importlib.reload(psd_Stats)
importlib.reload(psdClass)
importlib.reload(plot_specPsd)
importlib.reload(prep_specStats)

STATS_VERSION = '2Hz'

# prep_specStats.get_stats_REST_psds(
#     STAT_DATA_EXT_PATH = True,
#     STAT_LID_COMPARE = 'categs',
#     STATS_VERSION=STATS_VERSION,
#     PLOT_STATS=True,
#     MERGE_STNs=True,
#     STAT_PER_LID_CAT=True,
# )


# DEBUG ALTERNATIVE BASELINING FOR INVOLUNT (exclude no LID subs)

prep_specStats.get_stats_MOVE_psds(
    STAT_DATA_EXT_PATH = True,
    STAT_LID_COMPARE = 'categs',
    STATS_VERSION=STATS_VERSION,
    PLOT_STATS=True,
    STAT_PER_LID_CAT=True,
    SKIP_MOVES=['FREE'],  # FREEMOVE; FREENOMOVE; TAP; INVOLUNT
    MERGE_DYSK_SIDES=True,
    ALPHA=.01,
    ALT_BASELINE=True
)


In [ ]:
FIG_PATH = os.path.join(get_project_path('figures'),
                        'ft_exploration',
                        'data_v4.0_ft_v6',
                        'PSDs_state_specific')

cond_colors = {
    'nolid': 'green',
    'nolidbelow30': 'limegreen',
    'nolidover30': 'darkgreen',
    'alllid': 'blue', 'mildlid': 'orange',
    'moderatelid': 'red', 'severelid': 'purple',
}

Import 1-sec PSDs (rest-tap-dysk)


In [ ]:
importlib.reload(ephySel)
importlib.reload(psdClass)

RETURN_PSD_1sec = True

(
    PSDs_1s, BLs_1s
) = psdClass.get_allSpecStates_Psds(
    RETURN_PSD_1sec=RETURN_PSD_1sec,
    incl_free=False,
)




Import 1-sec PSDs (free)

In [ ]:
importlib.reload(psdClass)

RETURN_PSD_1sec = True

# TODO: INCLUDE ALTEARNTIVE BASELINING FOR FREE
# in plotting, but also still in STATS CREATION

(
    PSDs_free, BLs_free
) = psdClass.get_allSpecStates_Psds(
    RETURN_PSD_1sec=RETURN_PSD_1sec,
    incl_lidmove=False,
    incl_rest=False,
    incl_tap=False,
)


Plot REST or MOVEMENTS: Voluntary (TAP) / Involuntary (LID) versus CONTRA and IPSI-LATERAL Ephys

- movement selection has millisecond resolution

In [ ]:
importlib.reload(plot_ssd_PSDs)
importlib.reload(plot_specPsd)
importlib.reload(psdClass)
importlib.reload(ephySel)
importlib.reload(prep_specStats)

# PLOT DYSKINESIA STATES

%matplotlib inline
# %matplotlib qt
INCL_STATS = True
ALT_BASELINE = True

STATS_VERSION='2Hz'

for PLOT_SEL in ['REST', 'TAP', 'INVOLUNT', 'FREE']:
    # select state selection to plot
    if PLOT_SEL not in ['TAP', 'INVOLUNT']:
        print(f'\n...SKIP {PLOT_SEL}')
        continue
    print(f'PLOT {PLOT_SEL}, (incl STATs: {INCL_STATS})')
    
    # select matching PSD dict
    if PLOT_SEL == 'FREE':
        PSD_DICT = PSDs_free
        BL_DICT = BLs_free
    else:
        PSD_DICT = PSDs_1s
        BL_DICT = BLs_1s

    plot_specPsd.prep_and_plot_moveSpecPsd(
        PLOT_CONDITION=PLOT_SEL,
        PSD_DICT=PSD_DICT,
        BASELINE=BL_DICT,
        SAVE_PLOT=True,
        SHOW_PLOT=False,
        INCL_STATS=True,
        ALPHA=.01,
        ALT_MOVELID_BASELINE=ALT_BASELINE,
        STATS_VERSION=STATS_VERSION,
        STAT_PER_LID_CAT=True,
        STAT_LID_COMPARE='categs',
        MERGE_REST_STNS=True,
        MERGED_DYSK=True,
        ADD_TO_FIG_NAME='3001_altBL_a01_'
    )


### ### NOTES on DETAILED DATA SELECTION ### # 

- To Do:
    - plot unilat-dyskinesia in IPSI and CONTRA hemisphere (STN and ECoG)


# older scripts

tap overviews plotted in movement_psd_analysis.plot_overview_tap_detection

### 1a. Load pickled acc-data (and plot Taps)


### 1b. Load SSD data

load dataclass with features, labels, and acc

In [ ]:

# importlib.reload(ftProc)

# class created in ftProc, FeatLidClass()
# mean-RMS standardly zscored
import lfpecog_analysis.ft_processing_helpers as ftProc

featLabPath = os.path.join(utilsFiles.get_project_path('data'),
                           'prediction_data',
                           'featLabelClasses')
feats6 = utilsFiles.load_class_pickle(
    os.path.join(featLabPath, 'featLabels_ftv6_Cdrs_StnOnly.P'),
    convert_float_np64=True
)

### 2. Explore ACC-activity analysis

In [ ]:
clrs = list(plotHelp.get_colors('PaulTol').values())

In [ ]:
importlib.reload(accDerivs)

In [ ]:
# raw_rms = load_matching_acc_rms(sub='008', RMS_ZSCORE=True,
#                             FT_VERSION=feats6.FT_VERSION,
#                             DATA_VERSION=feats6.DATA_VERSION)

dat = accDerivs.get_raw_acc_traces(sub='008', side='left',
                        data_version=feats6.DATA_VERSION)

In [ ]:
# PLOT RAW-ACC VS FEAT-RMS AND TASK

sub = '008'

FS=16  # fontsize

# sub = 
# get rms per 10 seconds
rms = feats6.ACC_RMS[sub]
rms_t = feats6.FEATS[sub].index.values  # is in minutes

# raw_rms = load_matching_acc_rms(sub=sub, RMS_ZSCORE=False,
#                             FT_VERSION=feats6.FT_VERSION,
#                             DATA_VERSION=feats6.DATA_VERSION)
    
fig, ax = plt.subplots(1,1, figsize=(8, 4))

# plot signal vector magnitude
for side, col in zip(['left', 'right'],
                     ['blue', 'orangered']):
    dat = get_raw_acc_traces(sub=sub, side=side,
                             data_version=feats6.DATA_VERSION)
    acc_axes = ['ACC_' in c for c in dat.colnames]
    svm = movePrep.signalvectormagn(dat.data[:, acc_axes])
    ax.scatter(dat.times / 60, svm, facecolor='w', alpha=.3,
                edgecolor=col, s=20, label=f'raw ACC-{side}')
    
ax.set_ylabel('raw ACC-SVM (a.u.)', size=FS,
              color='k',weight='bold',)

ax2 = ax.twinx()

# plot windowed RMS
ax2.scatter(rms_t, rms, s=50,
            alpha=.5, color='darkorange',
            label='mean ACC-RMS')
ax2.set_ylabel('Windowed ACC-RMS\nz-scored (a.u.)',
               size=FS, color='darkorange',
               weight='bold',)

# shade tasks
for i_task, task in enumerate(['rest', 'tap', 'free']):
    task_sel = dat.data[:, task_col] == task
    ax.fill_between(x=(dat.times/60).astype(np.float64),
                    where=task_sel,
                    y1=0, y2=7e-6,
                    alpha=.3,
                    color=clrs[i_task*2],
                    # hatch='//',  facecolor='w',
                    label=task)

plt.title(f'Movement overview during experiment, sub-{sub}',
          size=FS, weight='bold', y=1.08,)

ax.legend(ncol=3, loc='upper center',
          frameon=False, fontsize=FS-4,
          bbox_to_anchor=(.5, 1.05))

for AX in [ax, ax2]:
    AX.tick_params(axis='both', size=FS, labelsize=FS)
    AX.spines['top'].set_visible(False)
    # AX.spines['right'].set_visible(False)

plt.tight_layout()

plt.savefig(os.path.join(get_project_path('figures'),
                         'protocol',
                         f'movement_ex_{sub}'),
            dpi=300, facecolor='w',)

plt.close()

In [ ]:
def load_matching_acc_rms(sub, FT_VERSION,
                          DATA_VERSION,
                          ACC_SIDE='mean',
                          RMS_ZSCORE = False,
                          WIN_LEN_sec=10,
                          WIN_OVERLAP=0.5):
    """
    duplicate of in-class function in FeatLidClass,
    use to get non-zscored-RMS
    """

    acc_rms_path = os.path.join(utilsFiles.get_project_path('results'),
                                'features',
                                f'SSD_feats_broad_{FT_VERSION}',
                                DATA_VERSION,
                                f'windows_{WIN_LEN_sec}s_'
                                f'{WIN_OVERLAP}overlap')


    rms_fname = f'windowed_ACC_RMS_{sub}.json'
    rms_fpath = os.path.join(acc_rms_path, rms_fname)

    if rms_fname in os.listdir(acc_rms_path):
        with open(rms_fpath, 'r') as f:
            sub_rms = json.load(f)
        
        if ACC_SIDE == 'mean':
            rms = np.array(sub_rms['left']) + np.array(sub_rms['right'])
        else: rms = sub_rms[ACC_SIDE]

        if RMS_ZSCORE: rms = (rms - np.mean(rms)) / np.std(rms)

    else:
        rms = accDerivs.get_acc_rms_for_windows(
            sub=sub, acc_side=ACC_SIDE, Z_SCORE=RMS_ZSCORE,
            featClass=feats6, SAVE_RMS=True,)
    
    return rms

In [ ]:
rms = load_matching_acc_rms(sub='103', FT_VERSION='v6',
                            DATA_VERSION='v4.0',)

In [ ]:
rms.shape

### 3. Explore Movement vs PSD analysis

In [ ]:
FT_VERSION='v6'
DATA_VERSION = 'v4.0'
main_data_path = os.path.join(get_project_path('data'),
                              'merged_sub_data',
                              DATA_VERSION)

SUBS = load_ssd_fts.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                                       FT_VERSION=FT_VERSION)
print(len(SUBS))

In [ ]:
importlib.reload(prepTapPSD)

for sub in SUBS:
    
    print(f'start sub-{sub}')
    prepTapPSD.create_sub_movement_psds(
        sub=sub,
        data_version=DATA_VERSION,
        ft_version=FT_VERSION,
        states_to_save=['tap',])


In [ ]:
def unpack_list_of_lists(lists_to_unpack):

    new_list = [i for j in lists_to_unpack for i in j]

    return new_list

In [ ]:
colors = list(plotHelp.get_colors().values())

In [ ]:
importlib.reload(movePSD)

# psd_rest, psd_tap, tap_f = movePSD.load_movement_psds()
state_psds, tap_f = movePSD.load_movement_psds()


In [ ]:
state_psds.keys()

In [ ]:
importlib.reload(movePSD)


movePSD.plotPSD_rest_vs_tap(PSDs=state_psds,
                           freqs=tap_f,
                           data_version=data_version,
                           n_subs_incl=17,
                           fig_name='STN_PSDs_Tap_vs_Rest_vs_Free'
                        )

#### 1b) Import processed dataclass per subject, optionally merge ACC-data into EPHYS-df

In [ ]:
# # Import subjectData Classes with alligned preprocessed Data
# importlib.reload(read_data)
# importlib.reload(add_moveStates)
# importlib.reload(findTap)

# incl_accStates = True

# subData = {}

# for sub in ['012', '014']:

#     print(f'start {sub}')
#     # if sub == '008': continue

#     subData[sub] = read_data.subjectData(
#         sub=sub,
#         data_version='v2.3',
#         project_path=projectpath,
#     )

#     if incl_accStates:
        
#         accStates = run_tap_detect.runTapDetection(subData[sub])

#         for group in subData[sub].dtypes:

#             if 'lfp' or 'ecog' in group:

#                 print(f'adding acc-states for {sub}: {group}')
#                 newdf = add_moveStates.add_detected_acc_states(
#                     df=getattr(subData[sub], group).data,
#                     detectedMoves=accStates,
#                 )
#                 getattr(subData[sub], group).data = newdf

### 2) Develop and Visualise Movement State Detection


#### Run single Acc-State Detections


In [ ]:
importlib.reload(run_tap_detect)
importlib.reload(movePrep)
importlib.reload(findTap)

taplists = {}
for sub in ['012',  '014']:  # '008', '013',
    print(sub)
    taplists[sub] = run_tap_detect.runTapDetection(subData[sub])

#### Visualise Performance of Tap/Move-detection

In [ ]:

fonts=20

for sub in list(subData.keys()):

    for x0, x1 in zip(
        # [9, 42],
        # [10, 43]
        [5, 37,],
        [15, 42]
    ):

        fig, axes = plt.subplots(2, 1, figsize=(16, 8))

        for s, side in enumerate(['left', 'right']):

            acc_df = getattr(subData[sub], f'acc_{side}').data  # per side
            fs = getattr(subData[sub], f'acc_{side}').fs

            ax = movePrep.find_main_axis(
                acc_df.iloc[:, 1:4].values
            )
            svm = movePrep.signalvectormagn(
                acc_df.iloc[:, 1:4].values
            )

            axes[s].plot(
                acc_df['dopa_time'] / 60,
                acc_df.iloc[:, ax + 1],
                alpha=.4, label='uni-axis'
            )
            axes[s].plot(
                acc_df['dopa_time'] / 60,
                movePrep.signalvectormagn(
                    acc_df.iloc[:, 1:4].values
                ), alpha=.4, label='svm', c='r', ls='dotted',
            )

            axes[s].scatter(
                np.array([l[0] for l in taplists2[sub][f'{side}_tap_t']]) / 60,
                [.65e-6] * len(taplists2[sub][f'{side}_tap_t']),
                s=50, color='g', label='tap-start',
            )
            axes[s].scatter(
                np.array([l[-1] for l in taplists2[sub][f'{side}_tap_t']]) / 60,
                [.6e-6] * len(taplists2[sub][f'{side}_tap_t']),
                s=50, color='r', label='tap-end'
            )
            axes[s].scatter(
                np.array([m[0] for m in taplists2[sub][f'{side}_move_t']]) / 60,
                [.55e-6] * len(taplists2[sub][f'{side}_move_t']),
                s=50, color='orange', label='move-start'
            )
            axes[s].scatter(
                np.array([m[1] for m in taplists2[sub][f'{side}_move_t']]) / 60,
                [.5e-6] * len(taplists2[sub][f'{side}_move_t']),
                s=50, color='purple', label='move-end'
            )

            axes[s].set_xlim(x0, x1)
            axes[s].set_ylim(-1e-6, 1e-6)
            axes[s].set_ylabel(
                f'Acceleration\n{side.upper()}'
                    '\n(g, m/s/s)',
                size=fonts
            )
            axes[s].tick_params(labelsize=fonts - 4)

        axes[s].set_xlabel('Time (minutes to L-Dopa intake)', size=fonts)


        plt.suptitle(
            f'Subject {sub} -  bilateral '
            'Movement detection',
            x=.1, y=.96, ha='left',
            size=fonts+4
        )
        # remove duplicate legend labels
        handles, labels = plotHelp.remove_duplicate_legend(
            plt.gca().get_legend_handles_labels()
        )

        fig.legend(
            handles, labels,
            frameon=False, fontsize=fonts - 4, ncol=3,
            loc='center left', bbox_to_anchor = [.55, .95])
        
        plt.tight_layout()

        # plt.savefig(os.path.join(
        #     fig_dir, 'tapping_detection',
        #     f'sub{sub}_moveDetect_newBorders_min{x0}_{x1}'
        # ), dpi=150, facecolor='w',)

        plt.close()